In [9]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact

In [7]:
import matplotlib
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

In [8]:
ordered_verts = ['T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'L1', 'L2', 'L3', 'L4']

## Post-processing for midline finder

In [5]:
preds = np.load('../outputs/midline_finder_preds.npz')
ids, masks, labels = preds.values()
ids.shape, masks.shape, labels.shape

((46,), (46, 1, 512, 512), (46, 13))

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [13]:
@interact
def plot_mask(name=list(ids)):
    img = np.load(f'../midline_data/testing/slices/coronal/{name}.npy')
    print(img.shape)
    fig, ax = plt.subplots(1,1, figsize=(9, 9))
    ax.axis('off')
    n = list(ids).index(name)
    mask = sigmoid(masks[n, 0])
    norm_labels = np.round(sigmoid(labels[n]))
    levels = []
    for i in range(norm_labels.shape[0]):
        if norm_labels[i] == 1:
            levels.append(ordered_verts[i])
    #mask = np.where(mask < 0.5, np.nan, 1.0)
    #contours = measure.find_contours(mask, 0.55)
    ax.set_title(levels)
    # 0: MIP, 1: MIDLINE, 2:STD
    ax.imshow(img[..., 1])
    ax.imshow(mask, alpha=0.5, cmap='plasma')
    #print(contours)
#     for contour in contours:
#         ax.plot(contour[:, 1], contour[:, 0], linewidth=3)
        

interactive(children=(Dropdown(description='name', options=('04_06_2014_431_Sag', 'fr_553_LS_Sag', '16_05_2014…